## Ignore this section

TODOs:



[x] Feedback on loaded file: column names, number columns, instances.

[x] Infer Column Types

[x] Allow user to modify column types.

[ ] Fix bug: spaces introduced in categorical values list

[ ] Format text as Markdown for API Key message (Maybe have separate cell output markdown widget).

[ ] Modify column types with a dropdown instead (but then text for categorical?)


[ ] Infer Row ID attribute and/or allow user to set this. (radio button in Infer Column Types table?)

[ ] Set default target attribute (radio button in Infer Column Types Table?)

[ ] Add required meta-data fields (name and description? more?)

[ ] OpenML Logo :)


# CSV to OpenML helper

This notebook helps you upload a csv-file to OpenML.
To use this notebook, run it cell-by-cell.
Whenever the text prompts you to do something, do that before continuing in the notebook.

If you experience issues using this notebook, or have further questions, please [click here](https://github.com/PGijsbers/csv-to-openml/issues/new) to open an issue on Github.

In [39]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown

import csv
import io
import re
import numpy as np
import openml
import pandas as pd

In [40]:
# UI components that will be rendered in this notebook:
upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Select a csv file'
)

publish_button = widgets.Button(
    description='Publish dataset',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check',
    visible=False
)

In [3]:
data = None

def on_file_uploaded(input_):
    global data
    file_content = io.StringIO(upload_widget.data[0].decode())
    
    has_header = csv.Sniffer().has_header(file_content.read(1024))
    file_content.seek(0)
    
    data = pd.read_csv(file_content, header=0 if has_header else None)
    publish_button.visible=True

upload_widget.observe(on_file_uploaded, 'data')

Please provide select the CSV file you want to upload to OpenML:

In [4]:
upload_widget

FileUpload(value={}, accept='.csv', description='Select a csv file')

In [5]:
Markdown(f"The selected file has {len(data)} rows and {len(data.columns)} columns. "
         "Below is a preview of the first rows of your csv file.")

The selected file has 150 rows and 5 columns. Below is a preview of the first rows of your csv file.

In [6]:
data.head()

,A,B,C,D,Flower
0,1,3.5,1.4,0.2,Iris-setosa
1,9,3.0,1.4,0.2,Iris-setosa
2,7,3.2,1.3,0.2,Iris-setosa
3,6,3.1,1.5,0.2,Iris-setosa
4,0,3.6,1.4,0.2,Iris-setosa


Next, we are going to run some code to help with data annotation...

In [7]:
def infer_column_types(dataframe, max_unique_values=10):
    column_types = { }
    for column in dataframe.columns:
        if dataframe[column].nunique() <= max_unique_values:
            column_types[column] = set(dataframe[column].unique())
        elif np.issubdtype(dataframe[column].dtype, np.number):
            column_types[column] = 'numeric'
        else:
            column_types[column] = 'string'
    return column_types

In [8]:
column_types = infer_column_types(data)

In [9]:
def set_column_type_maker(column_widget):
    def set_column_type(change):
            if ', ' in change['new']:
                column_types[column_widget.value] = set(change['new'].split(','))
            else:
                column_types[column_widget.value] = change['new']
    return set_column_type

coltype_widgets = []
coltype_widgets.append(
    widgets.HBox(
        [
            widgets.Label(
                'Column Names',
                layout=widgets.Layout(width='300px')),
            widgets.Label(
                'Column Types',
                layout=widgets.Layout(width='300px')),
            widgets.Label(
                'Example Values',
                layout=widgets.Layout(width='300px')),
        ])
)

for i, column in enumerate(data.columns):
    column_name_widget = widgets.Text(
        value=column,
        layout=widgets.Layout(width='300px')
    )    
    def set_column_name(change):
        column_types[change['new']] = column_types[change['old']]
        del column_types[change['old']]
        data.rename(columns={change['old']: change['new']}, inplace=True)
    
    column_name_widget.observe(set_column_name, 'value')
    
    if isinstance(column_types[column], set):
        type_display_value = ', '.join([str(el) for el in column_types[column]])
    else:
        type_display_value = column_types[column]
    column_type_widget = widgets.Text(
        value=type_display_value,
        layout=widgets.Layout(width='300px')
    )    
    set_column_type = set_column_type_maker(column_name_widget)    
    column_type_widget.observe(set_column_type, 'value')
    
    example_values_widget = widgets.Text(
        value=', '.join([str(v) for v in data[column].head().values]),
        layout=widgets.Layout(width='300px')
    )    
    
    coltype_widgets.append(
        widgets.HBox(
        [
            column_name_widget,
            column_type_widget,
            example_values_widget
        ])
    )    

It's crucial for OpenML to know the *type* of data in each column.
Each feature should be one of:

 - A numeric feature. Examples: `car price` or `tree height`
 - A string (text). Examples: `sales text` or `tree name`
 - A nominal feature (can only take one of a set of unique values). Examples: `car color` (red, blue, ...) or `evergreen` (yes, no).
 
Based on the data found in our csv file, we inferred the the types for each column.

Below you will find a table, in the left column you will find the column names of your data.
You can edit the column names directly.

The middle column shows the column types.
The accepted values here are either `numeric`, `string` or a number of values separated with `', '` (please note the space after the comma).

The right column simply shows some values of the column for easy reference.
This column should not be edited.

Please check that the types are correct, and correct any mistakes.

In [10]:
widgets.VBox(coltype_widgets)

Double-check that the data, column names and data types look correct (if not, retrace steps above):

In [11]:
display(data.head())
display(column_types)

,A,B,C,D,Flower
0,1,3.5,1.4,0.2,Iris-setosa
1,9,3.0,1.4,0.2,Iris-setosa
2,7,3.2,1.3,0.2,Iris-setosa
3,6,3.1,1.5,0.2,Iris-setosa
4,0,3.6,1.4,0.2,Iris-setosa


{'A': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9},
 'B': 'numeric',
 'C': 'numeric',
 'D': 'numeric',
 'Flower': {'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}}

## Meta-data

Great! Now we just need some meta-data such as a name and description.
This meta-data makes it easier for others to find and understand your dataset.

In [12]:
name_widget = widgets.Text(
    placeholder='Name of the dataset',
    description='Name:',
    layout=widgets.Layout(width='900px')
)
description_widget = widgets.Textarea(
    placeholder=('A description of the dataset. '
                 'Include for example:\n'
                 ' - What is the domain of this dataset?\n'
                 ' - How was the dataset gathered?\n'
                 ' - What is the meaning of each feature?'
                ),
    description='Description:',
    layout=widgets.Layout(width='900px', height='120px')
)

creator_widget = widgets.Text(
    placeholder='Original creator(s) of the dataset',
    description='Creator(s):',
    layout=widgets.Layout(width='900px')
)

dataset_url_widget = widgets.Text(
    placeholder='URL to the dataset if it is also hosted elsewhere',
    description='Data URL:',
    layout=widgets.Layout(width='900px')
)

paper_url_widget = widgets.Text(
    placeholder='URL to the paper which introduced the dataset',
    description='Paper URL:',
    layout=widgets.Layout(width='900px')
)

citation_widget = widgets.Text(
    placeholder='Citation for inclusion in a bibliography',
    description='Citation:',
    layout=widgets.Layout(width='900px')
)

# https://help.data.world/hc/en-us/articles/115006114287-Common-license-types-for-datasets
licence_widget = widgets.Text(
    placeholder='Licence of the dataset, e.g. Public Domain, CC0, CC BY-NC',
    description='Licence:',
    layout=widgets.Layout(width='900px')
)

# For column names? or cells?
language_widget = widgets.Text(
    placeholder='Language(s) in the dataset',
    description='Paper URL:',
    layout=widgets.Layout(width='900px')
)

# Is this retrieval date? Data that the data itself was collected?
collection_date_widget = widgets.Text(
    placeholder='URL to the paper which introduced the dataset',
    description='Paper URL:',
    layout=widgets.Layout(width='900px')
)

widgets.VBox([name_widget, description_widget, creator_widget, dataset_url_widget, paper_url_widget, citation_widget, licence_widget])


## Uploading to OpenML

In [36]:
# openml.config.apikey = ''

In [37]:
# retrieve data: name_widget.value

In [41]:
if openml.config.apikey == '':
    key_text = widgets.Output()    
    need_api_key_text = """    
    We noticed you have not configured an API key for OpenML yet.
    To find your API key, log in on the [OpenML website](https://openml.org) ([register](https://www.openml.org/register) if needed)
    , go to your account page (click the avatar image on the top right) and click "API Authentication".
    """
    with key_text:
        display(Markdown(need_api_key_text))

    def set_openml_apikey(key):
        openml.config.apikey = key
        if re.fullmatch('[a-f0-9]{32}', key):
            text_and_input.close()
            publish_button.disabled = False

    key_input = interactive(set_openml_apikey, key='')
    key_input.kwargs_widgets[0].description = 'API Key:'

    text_and_input = widgets.VBox([key_text, key_input])
    # show 'need_api_key_text'
    display(text_and_input)
display(publish_button)

Button(description='Publish dataset', disabled=True, icon='check', style=ButtonStyle(), tooltip='Click me')